In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle
from gripper_controller import Gripper

In [3]:
jubilee = JubileeMotionController()

In [54]:
jubilee.reset()

Reconectando...


In [4]:
jubilee.home_all(mesh_mode_z=False)
jubilee.protect_tools(on=True)
jubilee.move_xyz_absolute(z=50)

In [24]:
camera = camera_tool(jubilee)

In [25]:
camera.install()

Desinstale a última ferramenta


In [9]:
jubilee.keyboard_controll()

X:   40.00 | Y:  130.00 | Z:  210.00 || Passo: 10  mm
Interrompido


In [41]:
import math

def tempo_deslocamento(distancia_mm, velocidade_mm_min, aceleracao=1500):

    v = velocidade_mm_min / 60  
    
    # distância necessária para acelerar até v
    d_acc = v**2 / (2 * aceleracao)
    
    if distancia_mm > 2 * d_acc:
        # perfil trapezoidal
        d_const = distancia_mm - 2 * d_acc
        tempo_total = 2 * (v / aceleracao) + (d_const / v)
    else:
        # perfil triangular
        v_max_real = math.sqrt(aceleracao * distancia_mm / 2)
        tempo_total = 2 * (v_max_real / aceleracao)
    
    return tempo_total


In [53]:
import os
import time

velocidades = [5000,10000,18000]
distancia_total = 0

pos1 = (50.0, 130.0, 210.0)
pos2 = (270.0, 390.0, 210.0)

n_repeticoes = 300

for velocidade in velocidades:
    jubilee.home_xy()
    jubilee.move_xyz_absolute(z=pos1[2])
    time.sleep(20)

    pasta = f"Imagens/velocidade_{velocidade}"
    os.makedirs(pasta, exist_ok=True)

    distancia_total = 0  
    video = 3

    pos_atual = jubilee.position

    dx = pos2[0] - pos1[0]
    dy = pos2[1] - pos1[1]
    dz = pos2[2] - pos1[2]

    corr_x = pos_atual[0] - pos1[0]
    corr_y = pos_atual[1] - pos1[1]
    corr_z = pos_atual[2] - pos1[2]

    dx_corrigido = dx + corr_x
    dy_corrigido = dy + corr_y
    dz_corrigido = dz + corr_z

    distancia = round((dx_corrigido**2 + dy_corrigido**2 + dz_corrigido**2)**0.5, 2)

    tempo_movimento = tempo_deslocamento(distancia, velocidade) + 1

    for i in range(n_repeticoes): 
        print(f"Iteração:{i}, velocidade:{velocidade}")

        jubilee.move_xyz_absolute(x=pos1[0], y=pos1[1], z=pos1[2], velocity=velocidade)
        time.sleep(tempo_movimento)
        distancia_total += distancia
        for j in range(3):
            camera.photo(os.path.join(pasta, f"posicao_0_distancia_{distancia_total}_e_{i}_t{j}.png"), video_index=video)
            time.sleep(2)

        distancia = round(((pos2[0]-pos1[0])**2 + (pos2[1]-pos1[1])**2)**0.5, 2)
        distancia_total += distancia
        tempo_movimento = tempo_deslocamento(distancia, velocidade) + 1

        jubilee.move_xyz_absolute(x=pos2[0], y=pos2[1], z=pos2[2], velocity=velocidade)
        time.sleep(tempo_movimento)
        for j in range(5):
            camera.photo(os.path.join(pasta, f"posicao_1_distancia_{distancia_total}_e_{i}_t{j}.png"), video_index=video)
            time.sleep(2)


Iteração:0, velocidade:5000
Iteração:1, velocidade:5000
Iteração:2, velocidade:5000
Iteração:3, velocidade:5000
Iteração:4, velocidade:5000
Iteração:5, velocidade:5000
Iteração:6, velocidade:5000
Iteração:7, velocidade:5000
Iteração:8, velocidade:5000
Iteração:9, velocidade:5000
Iteração:10, velocidade:5000
Iteração:11, velocidade:5000
Iteração:12, velocidade:5000
Iteração:13, velocidade:5000
Iteração:14, velocidade:5000
Iteração:15, velocidade:5000
Iteração:16, velocidade:5000
Iteração:17, velocidade:5000
Iteração:18, velocidade:5000
Iteração:19, velocidade:5000
Iteração:20, velocidade:5000
Iteração:21, velocidade:5000
Iteração:22, velocidade:5000
Iteração:23, velocidade:5000
Iteração:24, velocidade:5000
Iteração:25, velocidade:5000
Iteração:26, velocidade:5000
Iteração:27, velocidade:5000
Iteração:28, velocidade:5000
Iteração:29, velocidade:5000
Iteração:30, velocidade:5000
Iteração:31, velocidade:5000
Iteração:32, velocidade:5000
Iteração:33, velocidade:5000
Iteração:34, velocidade:

KeyboardInterrupt: 